## Exploratory Data Analysis

Exploratory data analysis (EDA) involves taking a first look at a datasets and summarising its salient characteristics using tables and graphics. It is (or should be) the stage before testing hypotheses and can be useful in informing hypotheses. In this chapter, we'll look at a few options for EDA using code.

To show how to do EDA using code, we will need a dataset to explore. We'll use the Grinell housing dataset, which covers houses sold between 2005 and 2015 in Grinnell, Iowa. It was assembled by a local estate agent, Matt Karjalahti.

First, let's import a couple of essential packages:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from itertools import islice

plt.style.use('plot_style.txt')

Now let's grab the data.

In [ ]:
df = pd.read_csv("https://vincentarelbundock.github.io/Rdatasets/csv/Stat2Data/GrinnellHouses.csv", index_col=0)
df.head()

There are some things we should do even before an exploratory data analysis; we should make sure we've cast the columns to the right types and converted any that need converting. Even a quick look at the data provided by `.head()` shows that the `Date` column is in a weird format, the number of `Baths` is a floating point number rather than an integer (is it possible to have half a bathroom? Maybe, but it doesn't sound very private), and there are some NaNs in there too.

Fortunately, there is guidance attached to the dataset as to what the Date column really means: it's just a counter of days that encodes the date of sale that begins with 1st Jan 2005=16436. The documents also tell us that the final year is 2015. We'd really like to have a more usable datetime column than this so let's sort this out first. Ideally, we'd like to *map* the `Date` column into a new `datetime` column that turns 16436 into 1st Jan 2005, 16437 into 2nd Jan 2005, and so on, until we have a mapping for every date between the first date in the dataset and the last. 

Our solution to create this mapping will involve a couple of steps. First, we'll use `pd.date_range` to get all of the dates from 1st Jan 2005 for a number of periods. We'll determine the number of periods from the maximum value of the `Date` column (which is the last date represented) minus the start counter, which we know is 16436. To create a mapping from the coded numbers to the dates, we'll use the `zip` function, which pairs two sets of numbers together, and the `dict` function so that we get key, value pairs that we can conveniently feed into a dataframe.

Here's the first step, creating a dictionary mapping:

In [ ]:
start_code = 16436
end_code = df['Date'].max() + 1 # +1 because of how ranges are computed; we want to *include* the last date

datetime_dict = dict(zip(range(start_code, end_code),
                               pd.date_range(start='2005/01/01', periods=end_code-start_code)))

# Look at first few dict entries:
list(islice(datetime_dict.items(), 5))

Alright, now we want to use this new datetime format. We use an `apply` at the row level:

In [ ]:
df['datetime'] = df['Date'].apply(lambda x: datetime_dict[x])
# Check the biggest rows by datetime
df.nlargest(5, columns=['datetime'])

An extra column containing datetime has been added and, looking at the max rows, we can see that it does indeed run all the way to 2015 as expected from the documentation of the dataset.

Okay, now we want to sort out the data type isses we saw earlier. But let's just check they're as bad as we think using `df.info()`

In [ ]:
df.info()

**pandas** read most of them in sensibly, but not *all* of them, so let's adjust the ones we need to:

In [ ]:
df = df.assign(Address = df['Address'].astype('string'),
               Bedrooms = df['Bedrooms'].astype(int),
               Baths= df['Baths'].astype(int),
               OrigPrice=df['OrigPrice'].astype(float),
               ListPrice=df['ListPrice'].astype(float),
               SalePrice=df['SalePrice'].astype(float))
df.info()

Now we are ready to do some exploratory data analysis! Having to do some initial variable type cleaning is a *normal* and *unavoidable* part of data analysis, especially when reading in from a format like CSV (which does not preserve data type but has great interoperability across systems). It's important and what we've seen here is a typical pattern.

But now we can start the exploratory data analysis!

## EDA using **pandas** built-in methods

**pandas** has some great options for built-in EDA; in fact we've already seen one of them, `df.info()` which, as well as reporting datatypes and memory usage, also tells us how many observations in each column are 'truthy' rather than 'falsy', ie how many have non-null values.

### Tables

A small step beyond `df.info` to get tables is to use `df.describe()` which, if you have mixed datatypes that include floats, will report some basic summary statistics:

In [ ]:
df.describe()

Although helpful, that sure is hard to read! We can improve this by using the `round()` method too:


In [ ]:
sum_table = df.describe().round(1)
sum_table

Now, although this is very basic, let's say you wanted to include it as a table of summary statistics in a paper. This is just a dataframe so you can export it just like you can *any* pandas dataframe. This includes options like `.to_excel`. For inclusion in a paper or report, you're most likely to want to report it as text:

In [ ]:
print(sum_table.to_string())

as Markdown:


In [ ]:
print(sum_table.to_markdown())

as HTML:

In [ ]:
print(sum_table.to_html())

or as latex:

In [ ]:
print(sum_table.to_latex(caption='Summary stats from EDA'))

And remember, with all of these, you can write them to file using the following structure:

```python

open('sum_stats_table.tex', 'w').write(sum_table.to_latex(caption='Summary stats from EDA'))

```

Of course, the stats provided in this pre-built table are not very customised. So what do we do to get the table that we actually want? Well, the answer is to draw on the contents of the previous data chapters, particularly the introduction to data analysis. Groupbys, merges, aggregations: use all of them to produce the EDA table that you want.

If you're exploring data, you might also want to be able to read everything clearly and see any deviations from what you'd expect quickly. **pandas** has some built-in functionality that styles dataframes to help you.

Here's an example that highlights some ways of styling dataframes, making use of several features such as: changing the units (`lambda` function), unstacking into a wider format (`unstack`), fill NaNs with unobtrusive strings (`.fillna('-')`), removing numbers after the decimal place (`.style.set_precision(0)`), and adding a caption (`.style.set_caption`).

In [ ]:
(df.groupby(['YearSold', 'Baths'])
   .mean()['SalePrice']
   .apply(lambda x: x/1e3)
   .unstack()
   .fillna('-')
   .style
   .set_precision(0)
   .set_caption('Sale price (thousands)')
)

Although a neater one than we've seen, this is still a drab table of numbers. The eye is not immediately drawn to it!

To remedy that, let's take a look at another styling technique: the use of colour. Let's say we wanted to make a table that showed a cross-tabulation between year and number of bathrooms. Naturally, we'll use `pd.crosstab` but we'll ask that the values that appear in the table (counts) be lit up with a heatmap:

In [ ]:
pd.crosstab(df['Baths'], df['YearSold']).style.background_gradient(cmap='plasma')

Here are a couple of other styling tips for dataframes.

In [ ]:
df.set_index('datetime').groupby(pd.Grouper(freq='3M'))['SalePrice'].mean().plot()

Coded value for date of sale (Jan 1, 2005=16436)